In [230]:
import mne
import glob
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal
from scipy.fft import fftshift
from bids import BIDSLayout
from util.io.ffr import *

### Import and clean data

In [118]:
spectrums = pd.read_csv('diff_spectrums.csv', sep = '\t')

In [222]:
spectrums

,sub,left_chs,x,y,freq,stim,stream,attended,dB_diff
0,9.0,AF3,175.0,366.0,130.0,130.0,l,True,3.262019
1,9.0,C1,182.0,245.0,130.0,130.0,l,True,0.063900
2,9.0,C3,144.0,245.0,130.0,130.0,l,True,2.720311
3,9.0,C5,102.0,245.0,130.0,130.0,l,True,4.814894
4,9.0,T7,63.0,245.0,130.0,130.0,l,True,4.795095
...,...,...,...,...,...,...,...,...,...
19435,40.0,P7,95.0,150.0,280.0,280.0,r,False,-4.462605
19436,40.0,PO3,174.0,121.0,280.0,280.0,r,False,0.766769
19437,40.0,PO7,130.0,114.0,280.0,280.0,r,False,-0.660786
19438,40.0,TP7,71.0,195.0,280.0,280.0,r,False,-2.714720


In [119]:
# Clean data

# Drop subs with missing channels
for sub in np.unique(spectrums['sub']):
    obs = len(spectrums[spectrums['sub'] == sub]) # 936 for sub 4, 972 for all other subs
    if obs != 972:
        print(f'Dropping sub {sub}, obs = {obs}')
        spectrums = spectrums[spectrums['sub'] != sub]
        spectrums = spectrums[~spectrums['sub'].isnull()] 

Dropping sub 4.0, obs = 936
Dropping sub nan, obs = 0


In [141]:
# Check data is in the right shape
n_subs = len(spectrums['sub'].unique()) 
n_chans = len(spectrums['left_chs'].unique()) 
n_stims = len(spectrums['stim'].unique())
n_streams = len(spectrums['stream'].unique())
n_attend = len(spectrums['attended'].unique()) 
n_freqs = len(spectrums['freq'].unique())

# Check
if n_subs != len(spectrums)/n_chans/n_stims/n_attend/n_freq/n_streams: # n_epochs across all subs
    raise ValueError('Incorrect number of subjects!')

### Reshape for permutation test

#### Get channel adjacency

In [236]:
# Lift epochs.info object from a random sub
sub = '14'
BIDS_ROOT = '../data/bids'
epochs = read_epochs(sub, 'clean')
left_channels = ['AF3', 'C1','C3','C5','T7','CP1','CP3','CP5','F1','F3','F5','F7','FC1','FC3','FC5','FT7','FT9','Fp1','O1','P1','P3','P5','P7','PO3','PO7','TP7','TP9']
epochs = epochs.pick(left_channels)

# Apply mne.channels.find_ch_adjacency
ch_adjacency, ch_names = mne.channels.find_ch_adjacency(epochs.info, ch_type = 'eeg')
ch_adjacency

Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 27


<27x27 sparse matrix of type '<class 'numpy.int64'>'
	with 169 stored elements in Compressed Sparse Row format>

#### Run test on each stim and each stream

In [237]:
stim_freqs = [130, 200, 280]
streams = ['l', 'r']

for stim_freq in stim_freqs:
    print(f'stim_freq: {stim_freq}')

    for stream in streams:
        print(f'stream: {stream}')

        # Select data
        stim_spectrums = spectrums[(spectrums['stim'] == stim_freq) & (spectrums['stream'] == stream)]

        # Average over attended and unattended
        stim_spectrums = stim_spectrums.groupby(['sub', 'freq', 'left_chs'], as_index = False)['dB_diff'].mean()

        # Check
        if n_subs != len(stim_spectrums)/n_freqs/n_chans:
            raise ValueError('Incorrect number of subjects!')

        X = np.empty((n_subs, n_freqs, n_chans))

        # Index into every single value and add to a matrix, so I know exactly where each value is
        for i, sub in enumerate(np.unique(stim_spectrums['sub'])):
            print(f'sub: {sub}')
            for j, freq in enumerate(np.unique(stim_spectrums['freq'])):
                for k, ch in enumerate(np.unique(stim_spectrums['left_chs'])):
                    value = stim_spectrums['dB_diff'][(stim_spectrums['sub'] == sub) &
                                         (stim_spectrums['freq'] == freq) &
                                         (stim_spectrums['left_chs'] == ch)]
                    X[i, j, k] = value

        print(np.shape(X))

        F_obs, clusters, cluster_p_values, H0 = mne.stats.permutation_cluster_test(X, adjacency = ch_adjacency)
        print(cluster_p_values)

stim_freq: 130
stream: l
sub: 5.0
sub: 6.0
sub: 8.0
sub: 9.0
sub: 11.0
sub: 14.0
sub: 24.0
sub: 25.0
sub: 29.0
sub: 30.0
sub: 31.0
sub: 33.0
sub: 35.0
sub: 38.0
sub: 39.0
sub: 40.0
sub: 41.0
sub: 42.0
sub: 43.0
(19, 3, 27)
Using a threshold of 1.882603
stat_fun(H1): min=0.185536 max=1.542308
Running initial clustering …
Found 0 clusters
[]
stream: r
sub: 5.0
sub: 6.0


/tmp/ipykernel_32467/4129936998.py:34: RuntimeWarning: Ignoring argument "tail", performing 1-tailed F-test
  F_obs, clusters, cluster_p_values, H0 = mne.stats.permutation_cluster_test(X, adjacency = ch_adjacency)
/tmp/ipykernel_32467/4129936998.py:34: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  F_obs, clusters, cluster_p_values, H0 = mne.stats.permutation_cluster_test(X, adjacency = ch_adjacency)


sub: 8.0
sub: 9.0
sub: 11.0
sub: 14.0
sub: 24.0
sub: 25.0
sub: 29.0
sub: 30.0
sub: 31.0
sub: 33.0
sub: 35.0
sub: 38.0
sub: 39.0
sub: 40.0
sub: 41.0
sub: 42.0
sub: 43.0
(19, 3, 27)
Using a threshold of 1.882603
stat_fun(H1): min=0.548037 max=2.144584
Running initial clustering …
Found 1 cluster


/tmp/ipykernel_32467/4129936998.py:34: RuntimeWarning: Ignoring argument "tail", performing 1-tailed F-test
  F_obs, clusters, cluster_p_values, H0 = mne.stats.permutation_cluster_test(X, adjacency = ch_adjacency)


  0%|          | Permuting : 0/1023 [00:00<?,       ?it/s]

...

  0%|          | Permuting : 0/1023 [00:00<?,       ?it/s]

[0.22460938 0.5546875 ]
stream: r
sub: 5.0
sub: 6.0
sub: 8.0
sub: 9.0
sub: 11.0
sub: 14.0
sub: 24.0
sub: 25.0
sub: 29.0
sub: 30.0
sub: 31.0
sub: 33.0
sub: 35.0
sub: 38.0
sub: 39.0
sub: 40.0
sub: 41.0
sub: 42.0
sub: 43.0
(19, 3, 27)
Using a threshold of 1.882603
stat_fun(H1): min=0.410858 max=2.368500
Running initial clustering …
Found 2 clusters


/tmp/ipykernel_32467/4129936998.py:34: RuntimeWarning: Ignoring argument "tail", performing 1-tailed F-test
  F_obs, clusters, cluster_p_values, H0 = mne.stats.permutation_cluster_test(X, adjacency = ch_adjacency)


  0%|          | Permuting : 0/1023 [00:00<?,       ?it/s]

[0.08398438 0.40722656]


In [215]:
# # Don't separate by stream
# stim_freqs = [130, 200, 280]

# for stim_freq in stim_freqs:
#     print(f'stim_freq: {stim_freq}')
#     # Select data

#     stim_spectrums = spectrums[spectrums['stim'] == stim_freq]

#     # Average over attended and unattended
#     stim_spectrums = stim_spectrums.groupby(['sub', 'freq', 'left_chs', 'stream'], as_index = False)['dB_diff'].mean()

#     # Check
#     if n_subs != len(stim_spectrums)/n_freqs/n_chans/n_streams:
#         raise ValueError('Incorrect number of subjects!')

#     X = np.empty((n_subs, n_freqs, n_streams, n_chans))

#     # Index into every single value and add to a matrix, so I know exactly where each value is
#     for i, sub in enumerate(np.unique(stim_spectrums['sub'])):
#         print(f'sub: {sub}')
#         for j, freq in enumerate(np.unique(stim_spectrums['freq'])):
#             for k, stream in enumerate(np.unique(stim_spectrums['stream'])):
#                 for l, ch in enumerate(np.unique(stim_spectrums['left_chs'])):
#                     value = stim_spectrums['dB_diff'][(stim_spectrums['sub'] == sub) &
#                                          (stim_spectrums['freq'] == freq) &
#                                          (stim_spectrums['stream'] == stream) &
#                                          (stim_spectrums['left_chs'] == ch)]
#                     X[i, j, k, l] = value

#     print(np.shape(X))

#     F_obs, clusters, cluster_p_values, H0 = mne.stats.permutation_cluster_test(X)
#     print(cluster_p_values)

sub: 5.0
sub: 6.0
sub: 8.0
sub: 9.0
sub: 11.0
sub: 14.0
sub: 24.0
sub: 25.0
sub: 29.0
sub: 30.0
sub: 31.0
sub: 33.0
sub: 35.0
sub: 38.0
sub: 39.0
sub: 40.0
sub: 41.0
sub: 42.0
sub: 43.0
(19, 3, 2, 27)
Using a threshold of 1.882603
stat_fun(H1): min=0.185536 max=2.144584
Running initial clustering …
Found 1 cluster


/tmp/ipykernel_32467/2421113483.py:30: RuntimeWarning: Ignoring argument "tail", performing 1-tailed F-test
  F_obs, clusters, cluster_p_values, H0 = mne.stats.permutation_cluster_test(X)


  0%|          | Permuting : 0/1023 [00:00<?,       ?it/s]

[0.73632812]
sub: 5.0
sub: 6.0
sub: 8.0
sub: 9.0
sub: 11.0
sub: 14.0
sub: 24.0
sub: 25.0
sub: 29.0
sub: 30.0
sub: 31.0
sub: 33.0
sub: 35.0
sub: 38.0
sub: 39.0
sub: 40.0
sub: 41.0
sub: 42.0
sub: 43.0
(19, 3, 2, 27)
Using a threshold of 1.882603
stat_fun(H1): min=0.365580 max=2.676459
Running initial clustering …
Found 3 clusters


/tmp/ipykernel_32467/2421113483.py:30: RuntimeWarning: Ignoring argument "tail", performing 1-tailed F-test
  F_obs, clusters, cluster_p_values, H0 = mne.stats.permutation_cluster_test(X)


  0%|          | Permuting : 0/1023 [00:00<?,       ?it/s]

[0.88671875 0.36328125 0.47460938]
sub: 5.0
sub: 6.0
sub: 8.0
sub: 9.0
sub: 11.0
sub: 14.0
sub: 24.0
sub: 25.0
sub: 29.0
sub: 30.0
sub: 31.0
sub: 33.0
sub: 35.0
sub: 38.0
sub: 39.0
sub: 40.0
sub: 41.0
sub: 42.0
sub: 43.0
(19, 3, 2, 27)
Using a threshold of 1.882603
stat_fun(H1): min=0.410858 max=2.705616
Running initial clustering …
Found 5 clusters


/tmp/ipykernel_32467/2421113483.py:30: RuntimeWarning: Ignoring argument "tail", performing 1-tailed F-test
  F_obs, clusters, cluster_p_values, H0 = mne.stats.permutation_cluster_test(X)


  0%|          | Permuting : 0/1023 [00:00<?,       ?it/s]

[0.36523438 0.81640625 0.90820312 0.5625     0.63378906]
